In [1]:
import pandas as pd
import numpy as np

filepath = 'C:/Users/Tim/Desktop/lighthouse/w10/d1/'
file = 'Online_Retail.xlsx'

In [8]:
df = pd.read_excel(filepath+file)

In [3]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [12]:
df['StockCode']= df['StockCode'].astype(str)

In [13]:
customers = df["CustomerID"].unique().tolist()
len(customers)

4372

In [18]:
np.random.shuffle(customers)

In [19]:
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

In [20]:
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]

In [22]:
# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in customers_train:
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_train.append(temp)

In [27]:
# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in validation_df['CustomerID'].unique():
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

In [28]:
from gensim.models import Word2Vec 

In [29]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(purchases_train, progress_per=200)

model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(3642068, 3677500)

In [30]:
model.init_sims(replace=True)

In [31]:
model.init_sims(replace=True)

In [32]:
print(model)

Word2Vec(vocab=3167, size=100, alpha=0.03)


In [33]:
import matplotlib.pyplot as plt

In [34]:
import umap

cluster_embedding = umap.UMAP(n_neighbors=30, min_dist=0.0,
                              n_components=2, random_state=42).fit_transform(X)

plt.figure(figsize=(10,9))
plt.scatter(cluster_embedding[:, 0], cluster_embedding[:, 1], s=3, cmap='Spectral')

ModuleNotFoundError: No module named 'umap'

In [35]:
products = train_df[["StockCode", "Description"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

<ipython-input-35-fd224f1a6514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.drop_duplicates(inplace=True, subset='StockCode', keep="last")


In [36]:
products_dict['84029E']

['RED WOOLLY HOTTIE WHITE HEART.']

In [37]:
# similar products to one purchase
def similar_products(v, n = 5):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [38]:
similar_products(model['90019A'])

<ipython-input-38-99aba269da44>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  similar_products(model['90019A'])
<ipython-input-37-a479bc400d1b>:5: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  ms = model.similar_by_vector(v, topn= n+1)[1:]


[('SILVER M.O.P ORBIT DROP EARRINGS', 0.7773244976997375),
 ('AMBER DROP EARRINGS W LONG BEADS', 0.7699626088142395),
 ('FLOWER GLASS GARLD NECKL36"AMETHYST', 0.7610843181610107),
 ('GOLD/M.O.P PENDANT ORBIT NECKLACE', 0.7534050941467285),
 ('PINK BOUDICCA LARGE BRACELET', 0.7518506050109863)]

In [39]:
# similar products to multiple purchases
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [40]:
len(purchases_val[0])

30

In [41]:
similar_products(aggregate_vectors(purchases_val[0]))

<ipython-input-39-87e55b539b1d>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  product_vec.append(model[i])
<ipython-input-37-a479bc400d1b>:5: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  ms = model.similar_by_vector(v, topn= n+1)[1:]


[('JUMBO BAG APPLES', 0.8048266768455505),
 ('JUMBO BAG ALPHABET', 0.7826830148696899),
 ('JUMBO BAG VINTAGE LEAF', 0.7761577367782593),
 ('JUMBO BAG PEARS', 0.7754483819007874),
 ('LUNCH BAG ALPHABET DESIGN', 0.7490617632865906)]

In [42]:
similar_products(aggregate_vectors(purchases_val[0][-10:]))

<ipython-input-39-87e55b539b1d>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  product_vec.append(model[i])
<ipython-input-37-a479bc400d1b>:5: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  ms = model.similar_by_vector(v, topn= n+1)[1:]


[('JUMBO BAG VINTAGE CHRISTMAS ', 0.7315481305122375),
 ('JUMBO BAG VINTAGE DOILY ', 0.7257436513900757),
 ('JUMBO BAG PEARS', 0.6721333265304565),
 ('JINGLE BELL HEART DECORATION', 0.670067310333252),
 ('ZINC FOLKART SLEIGH BELLS', 0.6561737060546875)]